# 💡 Simulated Data Enrichment for Fraud Detection

In this notebook, we enrich a single `creditcard.csv` dataset by simulating semi-real-world customer and transaction context. The dataset originates from [Kaggle](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud/data), and this enrichment process simulates a semi-real-world task faced in the **payments industry** and **consulting environments**, where raw logs are often combined with user, merchant, and geographic metadata to drive business value.


Our motivation for doing this is to:
- showcase more nuanced SQL queries,  
- allow for a more realistic analysis, and
- showcase data wrangling & feature engineering abilities.  


In [7]:
# Import Libraries
import pandas as pd
import numpy as np
import sqlite3
from scipy.stats import norm
import os

# Set display settings
pd.set_option('display.max_columns', None)

# Set seed for reproducibility
seed = 12

In [8]:
# Confirm the current working directory
print("Current directory:", os.getcwd())

Current directory: /Users/chloestipinovich/Documents/Personal Programming Projects/Credit-Card-Fraud-Detection/notebooks


## 📥 Step 1: Load the Raw Transactions

We begin by reading the raw `creditcard.csv` into a pandas DataFrame. This file contains anonymized transaction records (amount, timestamp, and a fraud label).  We'll then pushing it into a local SQLite database as the `transactions` table.

In [9]:
# Load the dataset
df = pd.read_csv("../data/creditcard.csv")
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [11]:
# Connect to SQLite db
conn = sqlite3.connect('../data/fraud_detection.db')

## 🧍‍♂️📊 Step 2: Simulating Customer Profiles

**Design Logic**

- The dataset contains roughly **284,000 transactions over 2 days**.
- Assuming an average of approx 4 transactions per customer, we simulate **~70,000 unique customers**.
- **Age** is assigned randomly between 18 and 75 to reflect typical adult credit card users.
- Customers are assigned to one of four **geographic regions** (North, South, East, West) to allow for regional analysis.
- Customer tenure (`customer_since`) is randomly assigned over the past 5 years to reflect varying relationships with the bank.

In [12]:
# Set a random seed
np.random.seed(seed)

num_customers = 70000

# Generate customer IDs
customer_ids = np.arange(1, num_customers + 1)

# Random ages between 18 and 85
ages = np.random.randint(18, 86, size=num_customers)

# Randomly assign regions
regions = np.random.choice(['North', 'South', 'East', 'West'], size=num_customers)

# Generate customer_since dates between 2018-01-01 and 2023-05-21 (today-ish)
start_date = pd.to_datetime('2018-01-01')
end_date = pd.to_datetime('2023-05-21')
date_range_days = (end_date - start_date).days

customer_since = start_date + pd.to_timedelta(np.random.randint(0, date_range_days, size=num_customers), unit='D')

# Create the DataFrame
customers = pd.DataFrame({
    'customer_id': customer_ids,
    'age': ages,
    'region': regions,
    'customer_since': customer_since
})

customers.head()

,customer_id,age,region,customer_since
0,1,45,West,2021-10-04
1,2,24,West,2022-04-13
2,3,20,South,2019-03-15
3,4,21,East,2021-11-04
4,5,85,East,2022-12-18


In [13]:
# upload this DataFrame to SQLite
customers.to_sql('customers', conn, if_exists='replace', index=False)

70000

## ⚙️ Step 3: Assigning Transactions and Adjusting Amounts

We will now assign the transactions from the original `df` to the customer profiles created in `customers`. Rather than simply assigning customer IDs randomly, we will allocate transactions based on customer age. Additionally, we will adjust the simulated transaction amounts according to both customer age and region, as detailed below.

Although this approach may not perfectly reflect real-world data, it adds depth and realism for demonstration purposes.

### 📈 **Assigning Transactions to Customers Based on Age Distribution**

Instead of assigning transactions uniformly, we model the likelihood of transactions using a **normal distribution** centered around age 40. This reflects the tendency for middle-aged customers to be more active card users. The result is:

- Higher transaction frequency for customers around age 40  
- Gradual tapering off for both younger and older customers

We calculate probability weights from the normal distribution and sample customer IDs for transactions based on these probabilities, resulting in a more realistic age-related transaction pattern.


In [14]:
# Assign transactions to customer profies depending on age

# Parameters for the normal distribution
mu = 40        # peak at age 40
sigma = 15     # spread (adjust to make curve wider/narrower)

# Calculate probabilities using normal PDF
customers['weight'] = norm.pdf(customers['age'], loc=mu, scale=sigma)

# Normalize weights to sum to 1 for sampling probabilities
customers['prob'] = customers['weight'] / customers['weight'].sum()

# Sample customer_id for each transaction with these probabilities
customer_ids = customers['customer_id'].values
probabilities = customers['prob'].values

np.random.seed(seed)
df['customer_id'] = np.random.choice(customer_ids, size=len(df), replace=True, p=probabilities)

### 💰 **Adjusting Transaction Amounts by Customer Age and Region**

To simulate realistic spending behavior:

- Customers **aged 35 and older** receive a **higher spend multiplier** (20–50% increase), reflecting increased purchasing power or financial maturity  
- Customers **under age 35** use a baseline multiplier with some variation  
- Spending is further adjusted using **region-based multipliers** to account for geographic differences in purchasing behavior

The final transaction amounts are scaled accordingly to reflect both demographic and regional influences.

In [15]:
# simulate transaction amounts that depend on customer age and location

# First, merge the customer info (age, region) into your transactions DataFrame for convenience:
df = df.merge(customers[['customer_id', 'age', 'region']], on='customer_id', how='left')

# Define spend multiplier based on age:
def age_spend_multiplier(age):
    if age >= 35:
        return np.random.uniform(1.2, 1.5)  # 20% to 50% higher spend, randomized
    else:
        return np.random.uniform(0.8, 1.1)  # baseline with some noise

# Define spend multiplier based on region (optional):
region_multipliers = {
    'North': 0.95,
    'South': 1.20,
    'East': 0.90,
    'West': 1.00
}

# Apply multipliers:
df['age_multiplier'] = df['age'].apply(age_spend_multiplier)
df['region_multiplier'] = df['region'].map(region_multipliers).fillna(1.0)

# Adjust transaction amounts
df['Amount_adjusted'] = df['Amount'] * df['age_multiplier'] * df['region_multiplier']

# Drop helper columns if you want clean data
df.drop(columns=['age_multiplier', 'region_multiplier', 'Amount_adjusted'], inplace=True)

df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class,customer_id,age,region
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0,10802,25,North
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0,51855,43,South
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0,18410,41,North
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0,37328,46,South
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0,1024,25,South


In [16]:
# Save updated df (transactions with customer information merged)
df.to_sql('transactions', conn, if_exists='replace', index=False)

284807